### Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/15 12:47:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/15 12:47:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/15 12:47:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/15 12:47:35 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/07/15 12:47:35 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()



+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| John| 30|        11| 29000|
|James| 29|         9| 26000|
| Anne| 28|         5| 19000|
|Paula| 23|         3| 21000|
|Lizzy| 20|         1| 14000|
| Alex| 22|         2| 19000|
+-----+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [7]:
output=featureassembler.transform(training)

In [8]:
output.show()

+-----+---+----------+------+--------------------+
| Name|age|Experience|Salary|Independent Features|
+-----+---+----------+------+--------------------+
| John| 30|        11| 29000|         [30.0,11.0]|
|James| 29|         9| 26000|          [29.0,9.0]|
| Anne| 28|         5| 19000|          [28.0,5.0]|
|Paula| 23|         3| 21000|          [23.0,3.0]|
|Lizzy| 20|         1| 14000|          [20.0,1.0]|
| Alex| 22|         2| 19000|          [22.0,2.0]|
+-----+---+----------+------+--------------------+



In [9]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [10]:
finalized_data=output.select("Independent Features","Salary")

In [11]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [30.0,11.0]| 29000|
|          [29.0,9.0]| 26000|
|          [28.0,5.0]| 19000|
|          [23.0,3.0]| 21000|
|          [20.0,1.0]| 14000|
|          [22.0,2.0]| 19000|
+--------------------+------+



In [12]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

25/07/15 12:47:38 WARN Instrumentation: [753de2c1] regParam is zero, which might cause numerical instability and overfitting.
25/07/15 12:47:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/07/15 12:47:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [13]:
### Coefficients
regressor.coefficients

DenseVector([-1000.0, 2000.0])

In [14]:
### Intercepts
regressor.intercept

36999.99999999878

In [15]:
train_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [22.0,2.0]| 19000|
|          [28.0,5.0]| 19000|
|          [29.0,9.0]| 26000|
|         [30.0,11.0]| 29000|
+--------------------+------+



In [16]:
test_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [20.0,1.0]| 14000|
|          [23.0,3.0]| 21000|
+--------------------+------+



In [17]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [18]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [20.0,1.0]| 14000|18999.999999999833|
|          [23.0,3.0]| 21000| 19999.99999999992|
+--------------------+------+------------------+



In [19]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(2999.9999999999563, 12999999.999999246)